In [1]:
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import matplotlib.pyplot as plt
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import glob
import pickle
import gcsfs

import mCDR_utils as prerun

fs = gcsfs.GCSFileSystem()

2025-04-23 21:19:57.819983: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 21:19:57.835114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-23 21:19:57.852601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-23 21:19:57.857953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 21:19:57.872283: I tensorflow/core/platform/cpu_feature_guar

In [6]:
# path to HR-CESM-testbed
esm_dir = 'gs://leap-persistent/abbysh/ncar_files/processed' 

# where to store dataframe from this notebook, for use in 02 notebook
xgb_data_dir = 'gs://leap-persistent/your_username' 

path_seeds = 'gs://leap-persistent/abbysh/pickles/random_seeds.npy'
xco2_path = 'gs://leap-persistent/abbysh/ncar_files/processed/xco2_datm_rcp8_smooth.zarr' 
socat_path = 'gs://leap-persistent/abbysh/ncar_files/processed/socat_mask_monthly_10th_2000-2022_lat_50_70.zarr' 
topo_path = 'gs://leap-persistent/abbysh/ncar_files/processed/topography_mask_10th.zarr'
lsmask_path = 'gs://leap-persistent/abbysh/ncar_files/processed/land_mask_10th.zarr'

In [7]:
date_range_start = '2020-02-01T00:00:00.000000000'
date_range_end = '2022-01-01T00:00:00.000000000'

dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS') + np.timedelta64(14, 'D')

In [8]:
N_time = len(dates)
        
df = prerun.create_inputs(esm_dir,dates,N_time,xco2_path,socat_path,topo_path,lsmask_path)

In [7]:
df

key_0  sss  sst  HMXL  combined_chl  \
latitude longitude time                                              
-90.0    -180.0    2020-02-01      2  NaN  NaN   NaN           NaN   
                   2020-03-01      3  NaN  NaN   NaN           NaN   
                   2020-04-01      4  NaN  NaN   NaN           NaN   
                   2020-05-01      5  NaN  NaN   NaN           NaN   
                   2020-06-01      6  NaN  NaN   NaN           NaN   
...                              ...  ...  ...   ...           ...   
 90.0     180.0    2021-09-01      9  NaN  NaN   NaN           NaN   
                   2021-10-01     10  NaN  NaN   NaN           NaN   
                   2021-11-01     11  NaN  NaN   NaN           NaN   
                   2021-12-01     12  NaN  NaN   NaN           NaN   
                   2022-01-01      1  NaN  NaN   NaN           NaN   

                               pco2_residual  pCO2SURF  socat_mask  mld_log  \
latitude longitude time                                                       
-90.0    -180.0    2020-02-01            NaN       NaN         0.0      NaN   
                   2020-03-01            NaN       NaN         0.0      NaN   
                   2020-04-01            NaN       NaN         0.0      NaN   
                   2020-05-01            NaN       NaN         0.0      NaN   
                   2020-06-01            NaN       NaN         0.0      NaN   
...                                      ...       ...         ...      ...   
 90.0     180.0    2021-09-01            NaN       NaN         0.0      NaN   
                   2021-10-01            NaN       NaN         0.0      NaN   
                   2021-11-01            NaN       NaN         0.0      NaN   
                   2021-12-01            NaN       NaN         0.0      NaN   
                   2022-01-01            NaN       NaN         0.0      NaN   

                               mld_anom  ...  chl_log_anom  sss_anom  \
latitude longitude time                  ...                           
-90.0    -180.0    2020-02-01       NaN  ...           NaN       NaN   
                   2020-03-01       NaN  ...           NaN       NaN   
                   2020-04-01       NaN  ...           NaN       NaN   
                   2020-05-01       NaN  ...           NaN       NaN   
                   2020-06-01       NaN  ...           NaN       NaN   
...                                 ...  ...           ...       ...   
 90.0     180.0    2021-09-01       NaN  ...           NaN       NaN   
                   2021-10-01       NaN  ...           NaN       NaN   
                   2021-11-01       NaN  ...           NaN       NaN   
                   2021-12-01       NaN  ...           NaN       NaN   
                   2022-01-01       NaN  ...           NaN       NaN   

                               sst_anom        T0        T1    A  \
latitude longitude time                                            
-90.0    -180.0    2020-02-01       NaN  0.852078  0.523416 -1.0   
                   2020-03-01       NaN  0.497513  0.867456 -1.0   
                   2020-04-01       NaN -0.012910  0.999917 -1.0   
                   2020-05-01       NaN -0.504961  0.863142 -1.0   
                   2020-06-01       NaN -0.873807  0.486273 -1.0   
...                                 ...       ...       ...  ...   
 90.0     180.0    2021-09-01       NaN -0.490029 -0.871706  1.0   
                   2021-10-01       NaN  0.004304 -0.999991  1.0   
                   2021-11-01       NaN  0.512371 -0.858764  1.0   
                   2021-12-01       NaN  0.869589 -0.493776  1.0   
                   2022-01-01       NaN  0.999852  0.017213  1.0   

                                          B             C  net_mask  \
latitude longitude time                                               
-90.0    -180.0    2020-02-01 -7.498799e-33  6.123234e-17       NaN   
                   2020-03-01 -7.498799e-33  6.123234e-17

In [9]:
## Save dataframe 
def save_clean_data(df, output_dir, dates):
    
    init_date = str(dates[0].year) + format(dates[0].month,'02d')
    fin_date = str(dates[-1].year) + format(dates[-1].month,'02d')
    
    fname = f"{output_dir}/MLinput_CESM_mon_.1x.1_{init_date}_{fin_date}.pkl"
        
    df.to_pickle(fname)
    print(f"save complete")

In [10]:
save_clean_data(df,xgb_data_dir,dates)

save complete
